# Bulge data small ingestion

In this notebook we show how to ingest the raw DECam images. We are using one of the five fields of the bulge data set from [Saha et al. (2019)](https://arxiv.org/abs/1902.05637). The field has been observed from Apr. 1 - 3, 2015, with 43 visits and 62 detectors. The raw data is available on `/project/stack-club/course_data/decam-bulge-rawdata`. Here we will ingest only one visit and one CCD per night. . 

The raw images occupy around 50 GB of disk space.






In [ ]:
import os
import numpy as np
import sqlite3
import matplotlib
import matplotlib.pyplot as plt
import lsst.daf.persistence as dafPersist
import lsst.afw.table as afwTable
import lsst.afw.display as afwDisplay

afwDisplay.setDefaultBackend('matplotlib') 



## Setting up the directories.

In [ ]:
# Raw data directory
DATA_DIR = '/project/stack-club/course_data/decam-bulge-rawdata/'

# Repo directory, we will create a Butler repository in the home directory
REPO_DIR = '/home/mrabus/DATA/DECAM_BULGE_SingleVisit/' 
CALIB_DIR = REPO_DIR + "CALIB"
RERUN_DIR = REPO_DIR + "rerun"

In [ ]:
#if repo directory does not exist create it

if not os.path.isdir(REPO_DIR):
    # Create the REPO_DIR and add the mapper.
    ! mkdir -p {REPO_DIR}
    ! echo "lsst.obs.decam.DecamMapper" > {REPO_DIR+"_mapper"}
    ! mkdir -p {CALIB_DIR}
    


## Following the standard procedure, we start by linking the reference catalogs to the respository directory.



In [ ]:
#args = '/project/shared/decamBulge/saha2/ref_cats ' + REPO_DIR
#! ln -s {args}

args = REPO_DIR+'/ref_cats'
!mkdir {args}

args = '/datasets/refcats/htm/v1/sdss-dr9-fink-v5b/ '+REPO_DIR+'/ref_cats/sdss'
! ln -s {args}

args = '/datasets/refcats/htm/v1/ps1_pv3_3pi_20170110/ '+REPO_DIR+'/ref_cats/ps1_pv3_3pi_20170110'
! ln -s {args}

args = '/datasets/refcats/htm/v1/gaia_DR1_v1/ '+REPO_DIR+'/ref_cats/gaia'
! ln -s {args}

### We have to ingest several calibration images like crosstalk, defects, flats, biases and the images itself:

In [ ]:
# ingesting crosstalk
args = REPO_DIR + ' --calib ' + CALIB_DIR + ' ' + DATA_DIR + 'DECam/crosstalk/'
! ingestCuratedCalibs.py {args}

In [ ]:
# ingesting defects
args = REPO_DIR + ' --calib ' + CALIB_DIR + ' ' + DATA_DIR + 'DECam/defects/'
! ingestCuratedCalibs.py {args}

In [ ]:
# ingesting raw DECam images
args = REPO_DIR + ' ' + DATA_DIR + "raw/c4d_*.fits.fz --mode=link"
! ingestImages.py {args} 

In [ ]:
#ingesting flat and bias images
args = REPO_DIR + ' --calib ' + CALIB_DIR + ' ' + DATA_DIR + \
        "cal/c4d_150403*.fits.fz --mode=link --validity 999 "
! ingestCalibs.py {args}

#Works only if I ingest one night of cal data????

### Processing single visit and one CCD for each night!

Now we are processing a single visit on each night:

night 150402, visit = 427320, CCD = 1

night 150403, visit = 427940, CCD = 1

night 150404, visit = 428626, CCD = 1



In [ ]:
args = REPO_DIR + " --calib " + CALIB_DIR + " --rerun " + RERUN_DIR + \
        " --id visit=427320 ccdnum=1 " 
! processCcd.py {args}

In [ ]:
args = REPO_DIR + " --calib " + CALIB_DIR + " --rerun " + RERUN_DIR + \
        " --id visit=427940 ccdnum=1 " 
! processCcd.py {args}

In [ ]:
args = REPO_DIR + " --calib " + CALIB_DIR + " --rerun " + RERUN_DIR + \
        " --id visit=428626 ccdnum=1 " 
! processCcd.py {args}

## After processing the images we can start with the butler and displaying images:

In [ ]:
# point Butler to the current rerun directory
butler = dafPersist.Butler(RERUN_DIR)

In [ ]:
good_visits = []

# show available data
metadata = butler.queryMetadata('calexp',['visit','ccd','filter'])
for dataset in metadata:
    dataId = {'visit': int(dataset[0]), 'ccd': int(dataset[1]), 'filter':dataset[2]}
    if butler.datasetExists('src', dataId=dataId):
        print(dataId)
        
        good_visits.append(dataId['visit'])


We have three visits on three different nights.
Next, plot for each visit calexp.

In [ ]:
CCD = 1

for ii,visit in enumerate(good_visits):

    calexp = butler.get('calexp', visit=int(visit), ccd=CCD)

    plt.figure(ii)
    display = afwDisplay.Display(frame=ii, backend='matplotlib')
    display.scale("linear", "zscale")
    display.mtv(calexp[500:1500,2500:3000])


All three visits on the different nights correspond to the same pointing. We see from the images, that for the last night the seeing was not good.

In [ ]:
#Create a source catalog dictionary for the three visits and print for each visit number of detected sources

srcCatalog = {}
for visit in good_visits:

    dataId={'visit': visit, 'ccd': 1} #This dataId exists
    srcCatalog[visit] = butler.get('src', dataId=dataId)
    print( 'visit {}, number of detections {}'.format(visit, len(srcCatalog[visit])) )
    print( 'visit {}, median effective PSF area {}'.format(visit, np.median(srcCatalog[visit]['base_PsfFlux_area'])) )


Because for the last night the seeing was not good, the number of detected objects is lower and the effective PSF area is larger.



In [ ]:
srcCatalog[427320].asAstropy().to_pandas().head()

Plot for each visit the detected source 



In [ ]:
markers = ['+', 'x', '2']

plt.figure(figsize=(10,10))

for ii, visit in enumerate(good_visits):

    plt.scatter(srcCatalog[visit]['coord_ra'], srcCatalog[visit]['coord_dec'], marker=markers[ii], label='{}'.format(visit))

plt.legend()
plt.xlabel('RA')
plt.ylabel('Dec')


It looks very crowdy. 